In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.autograd as autograd
device = 'cuda:2'
import torch
import numpy as np
# Set random seeds for reproducibility
#torch.manual_seed(1)
#np.random.seed(1)

In [2]:
class Problems():
    def dlX_disp(self):
        domain_xcoord = np.random.uniform(-self.nelx/(2*(self.nelm)),self.nelx/(2*(self.nelm)),(self.batch_size - self.dlX_fixed.shape[0] - self.dlX_force.shape[0],1))
        domain_ycoord = np.random.uniform(-self.nely/(2*(self.nelm)),self.nely/(2*(self.nelm)),(self.batch_size - self.dlX_fixed.shape[0] - self.dlX_force.shape[0],1))
        domain_coord = np.concatenate((domain_ycoord,domain_xcoord),axis = 1)
        coord = np.concatenate((self.dlX_fixed.cpu().detach().numpy(), self.dlX_force.cpu().detach().numpy()),axis = 0)
        coord = np.concatenate((coord, domain_coord),axis = 0)
        coord = torch.tensor(coord,dtype=torch.float32,requires_grad=True).to(device)
        return coord

class Cantilever_Beam_2D(Problems):
    def __init__(self, nelx, nely, xid, yid, vf):

        # Initialize geometry parameters
        self.xid = xid
        self.yid = yid
        self.nelx = nelx
        self.nely = nely
        self.nele = self.nelx*self.nely
        self.nelm = max(self.nelx,self.nely)
        self.volfrac = vf
        self.E0 = 1
        self.nu = 0.3
        
        self.batch_size = 25000

        self.alpha_init = 1
        self.alpha_max = 100
        self.alpha_delta = 0.5
        self.penal = 3.0
        
        c_y, c_x=np.meshgrid(np.linspace(-(self.nely)/(2*self.nelm),(self.nely)/(2*self.nelm),self.nely),
                                                np.linspace(-(self.nelx)/(2*self.nelm),(self.nelx)/(2*self.nelm),self.nelx)
                                                ,indexing='ij')
        self.dlX = np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 1).reshape([-1,2])
        c_y, c_x=np.meshgrid(np.linspace(-(self.nely)/(2*self.nelm),(self.nely)/(2*self.nelm),2*self.nely),
                                                np.linspace(-(self.nelx)/(2*self.nelm),(self.nelx)/(2*self.nelm),2*self.nelx)
                                                ,indexing='ij')
        self.dlXSS = np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 1).reshape([-1,2])
        self.V = (np.max(self.dlX[:,0])-np.min(self.dlX[:,0]))*(np.max(self.dlX[:,1])-np.min(self.dlX[:,1]))

        #Problem boundary condition
        fixed_voxel = np.zeros((self.nely,self.nelx))
        fixed_voxel[:,0] = 1.0
        fixed_voxel = fixed_voxel.reshape([self.nele,1])
        dlX_fixed = self.dlX[np.where(fixed_voxel == 1.0)[0],:]

        F = 0.1
        self.F_vector = torch.tensor([[F], [0.0]], dtype=torch.float32).to(device)
        self.force_voxel = np.zeros((self.nely,self.nelx)) 
        self.force_voxel[yid,xid] = 1
        force_voxel = self.force_voxel.reshape([self.nele,1])
        dlX_force = self.dlX[np.where(force_voxel == 1)[0],:]

        self.dlX = torch.tensor(self.dlX, dtype=torch.float32,requires_grad=True).to(device)
        self.dlXSS = torch.tensor(self.dlXSS, dtype=torch.float32,requires_grad=True).to(device)
        self.dlX_fixed = torch.tensor(dlX_fixed, dtype=torch.float32,requires_grad=True).to(device)
        self.dlX_force = torch.tensor(dlX_force, dtype=torch.float32,requires_grad=True).to(device)

    def analytical_fixed_BC(self,u,coord):
        u = u*2*(1/(1+torch.exp(-20*(coord[:,1:2]+0.5))) - 0.5)
        return u

In [3]:
class PINN():
    def __init__(self, problem, disp_model):
        self.problem = problem
        self.disp_model = disp_model
  
    def pinn_loss(self,xPhys_m, coord):
        u = self.disp_model(coord)
        u = self.problem.analytical_fixed_BC(u,coord)

        u1 = u[:,0:1]
        u0 = u[:,1:2]
        uy_xyz = torch.autograd.grad(outputs=u1, inputs=coord,
                                        grad_outputs=torch.ones_like(u1),
                                        create_graph = True, retain_graph = True)[0]
        ux_xyz = torch.autograd.grad(outputs=u0, inputs=coord,
                                        grad_outputs=torch.ones_like(u0),
                                        create_graph = True, retain_graph = True)[0]
                                        
        eps11 = ux_xyz[:,1]
        eps12 = 0.5 * ux_xyz[:,0] + 0.5 * uy_xyz[:,1]
        eps22 = uy_xyz[:,0]

        youngs_modulus = 1000
        poissons_ratio = 0.3
        lame_mu = youngs_modulus / (2.0 * (1.0 + poissons_ratio))
        lame_lambda = youngs_modulus * poissons_ratio / (1.0 - poissons_ratio**2)
        trace_strain = eps11 + eps22
        squared_diagonal = eps11 * eps11 + eps22 * eps22
        energy = 0.5 * lame_lambda * trace_strain * trace_strain + lame_mu * (squared_diagonal + 2.0 * eps12 * eps12)
        energy = energy.reshape(-1,1)*(xPhys_m**3.0)
        energy_c = energy
        energy_ans = self.problem.V*torch.mean(energy)
        force_l = torch.mean(torch.matmul(self.disp_model(self.problem.dlX_force),self.problem.F_vector))
        loss = (energy_ans - force_l)
        return loss, energy_c

In [4]:
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np

class TO_Net(nn.Module):
    def __init__(self):
        super(TO_Net, self).__init__()
        low_band = 0.0
        high_band = 35
        c_y, c_x=np.meshgrid(np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1]),
                                                    np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1])
                                                    ,indexing='ij')
        self.kernel1 = torch.tensor(np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 0),dtype=torch.float32).to(device)
        self.kernel1 = torch.nn.Parameter(self.kernel1.requires_grad_())
        self.weights1 = torch.zeros([self.kernel1.shape[1],1],dtype=torch.float32).to(device)
        self.weights1 = torch.nn.Parameter(self.weights1.requires_grad_())
            
    def forward(self, x):
        y = torch.sin(torch.matmul(x,1.0* self.kernel1 ) + torch.ones([1,self.kernel1.shape[1]]).to(device))
        y = torch.sigmoid(torch.matmul(y, self.weights1))
        return y
    
class Disp_Net(nn.Module):
    def __init__(self):
        super(Disp_Net, self).__init__()
        low_band = 0.0
        high_band = 35
        c_y, c_x=np.meshgrid(np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1]),
                                                    np.linspace([-high_band,low_band],[-low_band,high_band],10).reshape([-1]),
                                                    indexing='ij')
        self.kernel1 = torch.tensor(np.stack((c_y.reshape([-1]),c_x.reshape([-1])),axis = 0),dtype=torch.float32).to(device)
        self.kernel1 = torch.nn.Parameter(self.kernel1.requires_grad_())
        self.weights1 = torch.zeros([self.kernel1.shape[1],2],dtype=torch.float32).to(device)
        self.weights1 = torch.nn.Parameter(self.weights1.requires_grad_())
    def forward(self, x):
        y = torch.sin(torch.matmul(x,1.0* self.kernel1 ) + torch.ones([1,self.kernel1.shape[1]]).to(device))
        y = torch.matmul(y, self.weights1)
        return y

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from IPython import display
import numpy as np
import torch.profiler


class DMF_TONN():

    def __init__(self, problem, to_model, disp_model):
        self.problem = problem
        self.disp_model = disp_model
        self.to_model = to_model
        self.log_vf = []
        self.log_disp_loss = []
        self.log_c = []
        self.log_pinn_init_loss = []
        self.log_fec = []
        self.log_xPhys = []
        self.pinn = PINN(self.problem, self.disp_model)
        self.total_epoch = 0

        self.disp_optimizer = optim.Adam(self.disp_model.parameters(), lr=0.000005)
        self.to_optimizer = optim.Adam(self.to_model.parameters(), lr=0.002)
        #self.coord = problem.dlX_disp()
    '''
    def train_step_disp(self, xPhys_m, coord):
        loss, _ = self.pinn.pinn_loss(xPhys_m, coord)
        self.log_pinn_init_loss.append(loss.item())
        self.disp_optimizer.zero_grad()
        loss.backward()
        self.disp_optimizer.step()
'''
    def fit_disp_init(self):
        epochs = 1000
        for epoch in range(epochs):
            if epoch % 100 == 1:
                display.clear_output(wait=True)

            coord = self.problem.dlX_disp()
            xPhys = torch.ones(coord.shape[0], 1) * 0.5
            loss, energy_c = self.pinn.pinn_loss(xPhys.to(device), coord)
            self.log_pinn_init_loss.append(loss.item())
            self.disp_optimizer.zero_grad()
            loss.backward()
            self.disp_optimizer.step()

        xPhys = torch.ones(self.problem.dlX.shape[0], 1) * 0.5
        loss, energy_c = self.pinn.pinn_loss(xPhys.to(device), self.problem.dlX)
        self.c1 = energy_c
        self.c_0 = torch.mean(energy_c)

    def to_loss(self, coord):
        self.total_epoch = self.total_epoch+1
        xPhys_m = self.to_model(coord)
        alpha = min(self.problem.alpha_init + self.problem.alpha_delta * self.total_epoch, self.problem.alpha_max)
        _, energy_c = self.pinn.pinn_loss(xPhys_m, coord)
        class ComputeDeDrho(torch.autograd.Function):
            @staticmethod
            def forward(ctx, xPhys_m, energy_c, coord):
                # 将必要的模型信息保存到上下文中，以便反向传播时使用

                ctx.xPhys_m = xPhys_m
                ctx.coord = coord
                
                # 正向传播部分：计算能量
                #loss, energy_c = opt.pinn.pinn_loss(xPhys_m, coord)
                # 返回正向输出
                ctx.save_for_backward(xPhys_m,energy_c,coord)
                return energy_c

            @staticmethod
            def backward(ctx, denergy):
                # 获取正向传播时保存的中间结果
                xPhys_m, energy_c, coord = ctx.saved_tensors
                
                # 计算反向传播梯度
                # 使用 torch.autograd.grad 计算能量关于 xPhys_m 的梯度
                grad_energy = torch.autograd.grad(
                outputs=energy_c, 
                inputs=xPhys_m, 
                grad_outputs=denergy, 
                create_graph=True,  # 支持二阶梯度
                retain_graph=True
                )[0]
                gradients = grad_energy
                # 返回负梯度，剩余的输入梯度为零
                return -gradients, torch.zeros_like(energy_c), torch.zeros_like(coord)
        c = torch.mean(ComputeDeDrho.apply(xPhys_m, energy_c, coord))
        xPhys_dlX = self.to_model(self.problem.dlX)
        vf = torch.mean(xPhys_dlX)
        loss =  alpha*(vf / self.problem.volfrac - 1.0) ** 2 + 1 * c / self.c_0.detach() 
        print('Epoch:', self.total_epoch)
        print('Total Loss:', loss.item())
        print("c",c.item())
        self.log_c.append(c.item())
        self.log_vf.append(vf.item())
        self.log_xPhys.append(xPhys_dlX.detach())
        return loss

    def fit_disp(self, epochs=200):
        for i in range(epochs):
            coord = self.problem.dlX_disp()
            xPhys_m = self.to_model(coord)
            loss, _ = self.pinn.pinn_loss(xPhys_m, coord)
            self.log_disp_loss.append(loss.item())
            self.disp_optimizer.zero_grad()
            loss.backward()
            self.disp_optimizer.step()
    '''
    def fit_to(self, epochs):
        
        for epoch in range(epochs):
            with torch.profiler.profile(
            activities=[
                torch.profiler.ProfilerActivity.CPU,
                torch.profiler.ProfilerActivity.CUDA  # 如果有GPU
            ],
            record_shapes=True,
            profile_memory=True,
            with_flops=True,
            with_stack=False
        ) as prof:
                self.fit_disp(50)
                if epoch % 10 == 1:
                    display.clear_output(wait=True)
                coord = self.problem.dlX_disp()
                loss = self.to_loss(coord)
                self.to_optimizer.zero_grad()
                loss.backward()
                self.to_optimizer.step()
            print(prof.key_averages().table(
    sort_by="flops",  # 也可以试试 total_flops
    row_limit=20
))
'''
           
    
    def fit_to(self, epochs):
        self.time_pde=[]
        self.time_density=[]
        for epoch in range(epochs):
            t1= time.time()
            self.fit_disp(50)
            t2 = time.time()
            self.time_pde.append(t2-t1)
            if epoch % 10 == 1:
                display.clear_output(wait=True)

            coord = self.problem.dlX_disp()
            t1= time.time()
            loss = self.to_loss(coord)
            self.to_optimizer.zero_grad()
            loss.backward()
            self.to_optimizer.step()
            t2 = time.time()
            self.time_density.append(t2-t1)
    

In [6]:
nelx = 60
nely = 20

#User defined load location
xid = 59
yid = 19

#User defined target volume fraction
vf = 0.3
problem= Cantilever_Beam_2D(nelx,nely,xid,yid,vf)
to_model= TO_Net().to(device)
disp_model_h = Disp_Net().to(device)

In [7]:
opt = DMF_TONN(problem, to_model, disp_model_h)
opt.fit_disp_init()

In [8]:
import time
t1= time.time()
opt.fit_to(400)
t2 = time.time()

Epoch: 392
Total Loss: 0.8536800146102905
c 0.0081946961581707
Epoch: 393
Total Loss: 0.8755147457122803
c 0.008412725292146206
Epoch: 394
Total Loss: 0.8530815839767456
c 0.008203857578337193
Epoch: 395
Total Loss: 0.8625898957252502
c 0.008304636925458908
Epoch: 396
Total Loss: 0.865077018737793
c 0.008333859033882618
Epoch: 397
Total Loss: 0.8557503819465637
c 0.008241653442382812
Epoch: 398
Total Loss: 0.8799347877502441
c 0.00846766121685505
Epoch: 399
Total Loss: 0.8588742017745972
c 0.00825289636850357
Epoch: 400
Total Loss: 0.855959951877594
c 0.008221060037612915


In [12]:
xPhys_dlX = opt.to_model(opt.problem.dlXSS.to(device))

In [13]:
tt = np.reshape(xPhys_dlX.cpu().detach().numpy(),(2*opt.problem.nely,2*opt.problem.nelx))

In [14]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import spsolve

# 参数设置
nelx, nely = 120, 40       # 单元数量
E, nu = 1000, 0.3         # 杨氏模量和泊松比
rho = 0.5                 # 密度
penal = 3.0               # 惩罚因子（此处设为1，即线性）

# 平面应力条件下的单元刚度矩阵
def lk():
    E = 1000
    nu = 0.3
    k = np.array([1/2 - nu/6, 1/8 + nu/8, -1/4 - nu/12, -1/8 + 3*nu/8,
                  -1/4 + nu/12, -1/8 - nu/8, nu/6, 1/8 - 3*nu/8])
    KE = E / (1 - nu**2) * np.array([[k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
                                     [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
                                     [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
                                     [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
                                     [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
                                     [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
                                     [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
                                     [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]])
    return KE

KE = lk()

# 全局自由度编号
ndof = 2 * (nelx + 1) * (nely + 1)

# 节点编号
def node_ids(i, j):
    n1 = (nely + 1) * i + j
    return np.array([2 * n1, 2 * n1 + 1,
                     2 * (n1 + nely + 1), 2 * (n1 + nely + 1) + 1,
                     2 * (n1 + nely + 2), 2 * (n1 + nely + 2) + 1,
                     2 * (n1 + 1), 2 * (n1 + 1) + 1])

# 装配稀疏矩阵结构
iK = []
jK = []
sK = []
rho = tt.T[:,::-1]
for elx in range(nelx):
    for ely in range(nely):
        el = elx * nely + ely
        nodes = node_ids(elx, ely)
        rho_el = rho[elx, ely]
        for i in range(8):
            for j in range(8):
                iK.append(nodes[i])
                jK.append(nodes[j])
                sK.append((rho_el ** penal) * KE[i, j])
'''
for elx in range(nelx):
    for ely in range(nely):
        el = elx * nely + ely
        nodes = node_ids(elx, ely)
        for i in range(8):
            for j in range(8):
                iK.append(nodes[i])
                jK.append(nodes[j])
                sK.append((rho ** penal) * KE[i, j])
'''
# 生成稀疏刚度矩阵
K = coo_matrix((sK, (iK, jK)), shape=(ndof, ndof)).tocsc()

# 载荷向量
F = np.zeros(ndof)
# 右下角节点施加载荷
frc_node = (nelx) * (nely + 1) + 0
F[2 * frc_node + 1] = -0.1  # 垂直向下的力
# 右边中间节点施加载荷
#mid_j = nely // 2
#frc_node = nelx * (nely + 1) + mid_j
#F[2 * frc_node + 1] = -0.1  # 垂直向下的力

# 边界条件（左边固定）
fixeddofs = []
for j in range(nely + 1):
    n = j
    fixeddofs += [2 * n, 2 * n + 1]

alldofs = np.arange(ndof)
freedofs = np.setdiff1d(alldofs, fixeddofs)

# 求解位移
U = np.zeros(ndof)
K_free = K[freedofs, :][:, freedofs]
F_free = F[freedofs]
U[freedofs] = spsolve(K_free, F_free)

# 计算 compliance
compliance = F @ U
print("Compliance:", compliance)

Compliance: 0.008203400143686467
